# Functions

In [11]:
import numpy as np
# import matplotlib.pyplot as plt
from scipy.io import loadmat
import os
%matplotlib inline

In [12]:
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

# Main

In [13]:
figmap = [[np.nan, 7,15,23,31,39,47,np.nan],
          [     1, 8,16,24,32,40,48,    55],
          [     2, 9,17,25,33,41,49,    56],
          [     3,10,18,26,34,42,50,    57],
          [     4,11,19,27,35,43,51,    58],
          [     5,12,20,28,36,44,52,    59],
          [     6,13,21,29,37,45,53,    60],
          [np.nan,14,22,30,38,46,54,np.nan]]
figmap = np.array(figmap)

In [14]:
# exp_folder = 'D:\\GoogleDrive\\retina\\Chou\'s data\\20210720\\SplitData\\'
exp_folder = 'D:\\GoogleDrive\\retina\\Troy\'s data\\20211008\\SplitData\\'
ensure_dir(exp_folder+'STA')
# ensure_dir(exp_folder+'MI\\Fig')

In [15]:
import warnings
warnings.simplefilter('ignore')

# for Fc in [2,4,7,10]:
#     for unit in [1,2]:
for filename in os.listdir(exp_folder):
    if filename.find('mat')==-1 and filename.find('mat')==-1:
        continue
    print(filename)
    annots = loadmat(exp_folder+filename)
    dt = 0.01
    sampling_rate = 20000
    TimeStamps =np.round(np.squeeze(annots['TimeStamps']))
    TimeStamps = TimeStamps.astype(int)
    x = annots['a_data'][0, TimeStamps[0]*sampling_rate:TimeStamps[1]*sampling_rate+1]
    x = x[::int(sampling_rate*dt)]
    x = x.astype(float)
    x = (x-np.mean(x))/np.std(x)
    T=np.arange(len(x))*dt
    XcT = T-T[int(len(T)/2)]

    window = [-1,1] # second
    timeshift = np.arange(window[0], window[1]+dt, dt)
    STA_cell = [0]*60
    for channelnumber in np.arange(1,61):
        rstate, _ = np.histogram(annots['Spikes'][0][channelnumber-1]-TimeStamps[0], np.append(0,T))
        if sum(rstate)/len(rstate)/dt <= 0.3 : continue

        nXcorr = np.correlate(x, rstate, 'same')/np.correlate(np.ones_like(x), rstate, 'same')
        STA_cell[channelnumber-1] = nXcorr[np.arange(np.where(XcT == window[0])[0],np.where(XcT ==  window[1])[0]+1)]
        # timeshift, Information = MIfunc(rstate, xstate, dt, window, perSpike = True)
        # Information_cell[channelnumber-1] = Information.copy()

    np.savez(exp_folder+'STA\\STA_'+filename[:-4]+'.npz', STA_cell = STA_cell, timeshift = timeshift)

OU_tau=500ms_cutoff=0_mean10_amp2.mat
OU_tau=500ms_cutoff=1_mean=10_C=0.05.mat
OU_tau=500ms_cutoff=1_mean=10_C=0.1.mat
OU_tau=500ms_cutoff=1_mean=10_C=0.2.mat
OU_tau=500ms_cutoff=1_mean=10_C=0.3.mat
OU_tau=500ms_cutoff=1_mean=13_C=0.05.mat
OU_tau=500ms_cutoff=1_mean=13_C=0.1.mat
OU_tau=500ms_cutoff=1_mean=13_C=0.2.mat
OU_tau=500ms_cutoff=1_mean=13_C=0.3.mat
OU_tau=500ms_cutoff=1_mean=1_C=0.05.mat
OU_tau=500ms_cutoff=1_mean=1_C=0.1.mat
OU_tau=500ms_cutoff=1_mean=1_C=0.2.mat
OU_tau=500ms_cutoff=1_mean=1_C=0.3.mat
OU_tau=500ms_cutoff=1_mean=4_C=0.05.mat
OU_tau=500ms_cutoff=1_mean=4_C=0.1.mat
OU_tau=500ms_cutoff=1_mean=4_C=0.2.mat
OU_tau=500ms_cutoff=1_mean=4_C=0.3.mat
OU_tau=500ms_cutoff=1_mean=7_C=0.05.mat
OU_tau=500ms_cutoff=1_mean=7_C=0.1.mat
OU_tau=500ms_cutoff=1_mean=7_C=0.2.mat
OU_tau=500ms_cutoff=1_mean=7_C=0.3.mat
WhiteNoise_mean=10_C=0.05.mat
WhiteNoise_mean=10_C=0.1.mat
WhiteNoise_mean=10_C=0.2.mat
WhiteNoise_mean=10_C=0.3.mat
WhiteNoise_mean=1_C=0.05.mat
WhiteNoise_mean=1_C=0.1

In [16]:
nXcorr

array([ 0.08527521,  0.05202055, -0.01976085, ...,  0.11276213,
        0.10263102,  0.17088268])

# 0513
diode_OU_tau=500ms_cutoff=0_mean10_amp2_07-Jul-2021_0.mat:  10-2 <br>
diode_OU_tau=500ms_cutoff=2_coeff=0.2_07-Jul-2021_0.mat:    2-0.4 <br>
diode_OU_tau=500ms_cutoff=2_coeff=0.6_07-Jul-2021_0.mat:    6-1.2 <br>
diode_OU_tau=500ms_cutoff=2_coeff=1.5_07-Jul-2021_0.mat:    15-3 <br>
diode_OU_tau=500ms_cutoff=2_mean10_amp2_07-Jul-2021_0.mat:  10-2 <br>
diode_OU_tau=500ms_cutoff=2_meaninten=10_07-Jul-2021_0.mat: 10-1.3677 <br>
diode_OU_tau=500ms_cutoff=2_meaninten=15_07-Jul-2021_0.mat: 15-1.3677 <br>
diode_OU_tau=500ms_cutoff=2_meaninten=5_07-Jul-2021_0.mat:  5-1.3677 <br>

# 0716
